과제 : work0716
titanic 데이터 // pclass : 등석 순위 / 요금 / 탑승 지역명 / 같이 탄 식구 등등...있음

생존자에 대한 분석.
어디에 탄 사람이 많이 살았을 까.
남 / 녀 생존율 -> 나이대
sibsp 형제자매 / parch 부모/자녀




데이터 처리
 - age : 연령대로 정리
 - alone, sibsp, parch ==> 서로 연관되어있음 확인
  -> family로 함께 탑승한 가족이 있는지 True or Flase화
 - 티켓값은 Southampton이 제일 비쌌고, 탑승자 또한 Southampton이 가장 많았다.
 - 


?ㅇ
  - 가족 10명? 11명이 같이 탄 이들이 있다.
  - 명단에 있는 7명의 형제자매들은 모두 사망했으며, 그들의 다른 가족들은 명단에도 없다.
  - 1~3등선 티켓들의 평균 금액은 각각 84, 20, 13이며. 최고치는 512, 73.5, 69.55이다. 갈수록 최고치 편차가 크며 모두의 티켓값이 다른 것을 보아 항공권처럼 계속 요금이 변동되는 듯 하다. 
  - 2,3등선의 티켓을 가장 비싸게 판 곳은 Southampton이고, 1등선은 비교적 평균치보다 높은 금액으로 구매한 사람이 많은 편인데, 그중 Cherbourg에서 가장 비싼 금액으로 팔았다.

  - Cherbourg에서 1등선 티켓을 가장 비싸게 산 사람 3명, 금액은 동일한데 서로 지인은 아닌듯 하다. 모두 산 사람 wow
   -> 3명 중 2명은 커플 확률이 있고, 1명은 아이와 함께 탄 느낌. Cherbourg에서 출발한 1등급 티켓 중에서는 10대 아이가 1명 있다. 
   

In [37]:
import pandas as pd
file_name = pd.read_csv('titanic.csv')
titaDF = pd.DataFrame(file_name)

In [38]:
titaDF

""" survived    : 생존
	pclass	    : 등급
    sex	        : 성별
    age         : 나이	
    sibsp	    : 함께 탑승한 형제자매
    parch	    : 함께 탑승한 부모자녀
    fare	    : 요금
    embarked	: 탑승 항구
    class	    : 등급 
    who	        : 성별
    adult_male	: 성인 남자?
    deck	    : 갑판 번호
    embark_town	: 탑승 항구 이름
    alive	    : 생존
    alone       : 혼자 탑승 했는가?
"""

' survived    : 생존\n\tpclass\t    : 등급\n    sex\t        : 성별\n    age         : 나이\t\n    sibsp\t    : 함께 탑승한 형제자매\n    parch\t    : 함께 탑승한 부모자녀\n    fare\t    : 요금\n    embarked\t: 탑승 항구\n    class\t    : 등급 \n    who\t        : 성별\n    adult_male\t: 성인 남자?\n    deck\t    : 갑판 번호\n    embark_town\t: 탑승 항구 이름\n    alive\t    : 생존\n    alone       : 혼자 탑승 했는가?\n'

In [39]:
## 1차 중복 삭제.
titaDF.drop(['survived','pclass','who','embarked','adult_male'], axis=1, inplace=True)
    # 중복내용 삭제 -> 생존여부 , 티켓등급, 탑승항구, 성인 남자 없애고
titaDF
tiDF = titaDF.copy()

In [40]:
# tiDF['parch'].max(), tiDF['sibsp'].max()

## 2차 bin 작업
 - class : First, Second, Third > 되어있었음. (굿)
 
 - age : 10대, 20대 10살 별로 분류할것. (완)
 - 형제자매 + 부모자녀 합쳐서 family 만들것도 고려해볼것 => titaDF가 원전 tiDF에 family True False로 분류해둠.
 - alone이랑 형제자매부모자녀랑만 상관관계가 있나 체크 => alone과 가족 True,Flase 완전히 일치. => 친구들과 탑승 구분 못함.


    * 가족 11명 같이 탔음.... (생사를 ,... 확인 해 말아.)
    * alone, sibsp, parch == 가족과 같이 탔는가에 대한 비슷한 류의 정보였다.

#### age 나이대별로 분류 (완)

In [41]:
# [2-1] age 분류
# tiDF['age'].max()   => 80./
# tiDF['age'].min()   => 0.42
titaDF['age'] = pd.cut(tiDF['age'], bins=[0,9,19,29,39,49,59,69,79,89],labels=['baby','teens','20s','30s','40s','50s','60s','70s','80s'])

#### tiDF 에서 family (True,False)화

##### alone 이랑 family가 동일한가 확인. (능력부족.. 따로따로)

In [42]:

# (tiDF['alone']) == (tiDF_fam[tiDF_fam['family']==0])
# alone_check = pd.concat([tiDF['alone'],tiDF_fam['family']])
tiDF['alone'].value_counts()

alone
True     537
False    354
Name: count, dtype: int64

In [43]:
# 가족이 아닌 다른 사람과만 온 사람. 없어?
titaDF[(titaDF['alone']==False) & ~(titaDF['sibsp']>0) & ~(titaDF['parch']>0)]

# 없네

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone


##### family화

In [44]:
# sibsp + parch 합쳐서 family 만들고 쟤네 둘은 삭제 
tiDF_fam = pd.DataFrame(tiDF['sibsp']+tiDF['parch'], columns=['family'])
tiDF_fam['family'] = pd.cut(tiDF_fam['family'], bins=[-1,0.9,10], labels=[False,True])
# 가족이 있는지 없는지로 새로 만듬.
tiDF_fam

,family
0,True
1,True
2,False
3,True
4,False
...,...
886,False
887,False
888,True
889,False


In [45]:
tiDF_fam['family'].value_counts()

family
False    537
True     354
Name: count, dtype: int64

In [46]:
# 형제자매 + 부모자녀 -> family로 합체

tiDF.drop(['sibsp','parch'], axis=1, inplace=True)
tiDF['family'] = tiDF_fam['family']
tiDF

,sex,age,fare,class,deck,embark_town,alive,alone,family
0,male,22.0,7.2500,Third,NaN,Southampton,no,False,True
1,female,38.0,71.2833,First,C,Cherbourg,yes,False,True
2,female,26.0,7.9250,Third,NaN,Southampton,yes,True,False
3,female,35.0,53.1000,First,C,Southampton,yes,False,True
4,male,35.0,8.0500,Third,NaN,Southampton,no,True,False
...,...,...,...,...,...,...,...,...,...
886,male,27.0,13.0000,Second,NaN,Southampton,no,True,False
887,female,19.0,30.0000,First,B,Southampton,yes,True,False
888,female,NaN,23.4500,Third,NaN,Southampton,no,False,True
889,male,26.0,30.0000,First,C,Cherbourg,yes,True,False


#### 요금, 탑승 항구, 등급 관계 (임시스탑)

In [47]:
"""
1등급 평균 가격
2등급 평균 가격
3등급 평균 가격
"""

tiDF_fare = tiDF[['embark_town','class','fare']]

stDF = tiDF[(tiDF['embark_town']=='Southampton') & tiDF['class']=='First']

st_DF = pd.concat([tiDF[(tiDF['embark_town']=='Southampton') & tiDF['class']=='First'],
                   tiDF[(tiDF['embark_town']=='Southampton') & tiDF['class']=='Second'],
                   tiDF[(tiDF['embark_town']=='Southampton') & tiDF['class']=='Third']],
                    axis=1, keys=['First','Second','Third']
)
st_DF

Empty DataFrame
Columns: [(First, sex), (First, age), (First, fare), (First, class), (First, deck), (First, embark_town), (First, alive), (First, alone), (First, family), (Second, sex), (Second, age), (Second, fare), (Second, class), (Second, deck), (Second, embark_town), (Second, alive), (Second, alone), (Second, family), (Third, sex), (Third, age), (Third, fare), (Third, class), (Third, deck), (Third, embark_town), (Third, alive), (Third, alone), (Third, family)]
Index: []

[0 rows x 27 columns]

In [48]:
Third = tiDF_fare[tiDF_fare['class']=='Third']
print(f"3등급 \n최소금액 : {Third['fare'].min()}, 최고금액 : {Third['fare'].max()}, 평균 금액 : {Third['fare'].mean()}, 중간 금액 : {Third['fare'].median()}")

3등급 
최소금액 : 0.0, 최고금액 : 69.55, 평균 금액 : 13.675550101832993, 중간 금액 : 8.05


#### 항구별 3등급 중 대가족
    - 892명 중 3등급 티켓은 총 491개.
    - 항구는 총 3개. Southampton(런던), Cherbourg(프랑스), Queenstown
    - 3등급 티켓탑승자가 제일 많이 탄 곳은 Southampton(런던), 총 353석



In [49]:
# 최소금액 : 0.0, 최고금액 : 69.55, 평균 금액 : 13.675550101832993, 중간 금액 : 8.05
Third['embark_town'].describe()

# 3등선 총 491명 / 총 3개의 항구에서 탑승한 이들이고, 가장 많이 탑승한 항구가 Southampton, 총 353명이 탑승한 듯 하다.

count             491
unique              3
top       Southampton
freq              353
Name: embark_town, dtype: object

In [50]:
Third[Third['embark_town']=='Southampton'].describe()
# 3등급 티켓러들 중 Southampton에서 탑승한 이들의 요금 정보.

# 353명의 티켓 가격은 평균 14.6이고 최저는 0, 최고는 69.55이다.
# 3/4의 사람들이 16.1 이하의 가격에 샀는데 최고치가 혼자 너무 튄다. 

,fare
count,353.000000
mean,14.644083
std,13.276609
min,0.000000
25%,7.854200
50%,8.050000
75%,16.100000
max,69.550000


In [51]:
# Southampton지역에서 60원 이상 낸 사람들 뽑아낸다.
titaDF[(titaDF['embark_town']=='Southampton') & (titaDF['fare']>=60) & (titaDF['class']=='Third')] 
# 3등급 티켓을 69.55에 산 사람들은 다 Southampton 사람들. 근데 다 가족이 있네 ? 

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
159,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
180,female,NaN,8,2,69.55,Third,NaN,Southampton,no,False
201,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
324,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
792,female,NaN,8,2,69.55,Third,NaN,Southampton,no,False
846,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
863,female,NaN,8,2,69.55,Third,NaN,Southampton,no,False


In [52]:
# 형제자매 8명 , 부모자식 2명이서 왓다는데 7명만 뜬다. 전체 항구로 검색.
titaDF[(titaDF['fare']>=60) & (titaDF['class']=='Third')]

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
159,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
180,female,NaN,8,2,69.55,Third,NaN,Southampton,no,False
201,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
324,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
792,female,NaN,8,2,69.55,Third,NaN,Southampton,no,False
846,male,NaN,8,2,69.55,Third,NaN,Southampton,no,False
863,female,NaN,8,2,69.55,Third,NaN,Southampton,no,False


In [53]:
# 부모찾아 삼만리
titaDF[(titaDF['parch']>=5)]
# 없다.
# 타이타닉 총 탑승자는 2224명, 명단은 총 892명, 11명의 대가족의 일부는 명단에 포함되지도 못했다.
# 당시 워낙 대가족이라 생존자 또는 항구의 티켓판매자가 기억해서 11명의 대가족이었다고 얘기가 나왔던건가?
# 솔직히 3배 값에 샀으면 기억할만햇다

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
13,male,30s,1,5,31.2750,Third,NaN,Southampton,no,False
25,female,30s,1,5,31.3875,Third,NaN,Southampton,yes,False
610,female,30s,1,5,31.2750,Third,NaN,Southampton,no,False
638,female,40s,0,5,39.6875,Third,NaN,Southampton,no,False
678,female,40s,1,6,46.9000,Third,NaN,Southampton,no,False
885,female,30s,0,5,29.1250,Third,NaN,Queenstown,no,False


#### 클래스별 평균 요금

In [54]:
# 클래스에 따른 평균 요금은 얼마인가 / 탑승 항구마다 금액이 다른가?
# 높은 클래스일수록 더 살았는가?

# tiDF_s = pd.DataFrame(index=tiDF['class'].T,columns=tiDF[['fare','embark_town']])
# 2차거름망 DF에서 다시 등급 요금 항구 생존여부 3차DF.
tiDF_s = pd.DataFrame(tiDF[['class','fare','embark_town','alive']])
tiDF_s.sort_values('class') # 클래스 별 정렬
#
# tiDF_s[tiDF_s[tiDF_s[tiDF_s['class']=='First']['alive']=='yes']]
print(f"총 생존자 : {tiDF_s[tiDF_s['alive']=='yes'].count()['alive']}, 1등선 생존자 : {tiDF_s[(tiDF_s['class']=='First') &(tiDF_s['alive']=='yes')].count()['alive']}, 2등선 생존자 : {tiDF_s[(tiDF_s['class']=='Second') &(tiDF_s['alive']=='yes')].count()['alive']}, 3등선 생존자 : {tiDF_s[(tiDF_s['class']=='Third') &(tiDF_s['alive']=='yes')].count()['alive']}")



총 생존자 : 342, 1등선 생존자 : 136, 2등선 생존자 : 87, 3등선 생존자 : 119


In [55]:
tiDF # 참고용 띄워놓기

,sex,age,fare,class,deck,embark_town,alive,alone,family
0,male,22.0,7.2500,Third,NaN,Southampton,no,False,True
1,female,38.0,71.2833,First,C,Cherbourg,yes,False,True
2,female,26.0,7.9250,Third,NaN,Southampton,yes,True,False
3,female,35.0,53.1000,First,C,Southampton,yes,False,True
4,male,35.0,8.0500,Third,NaN,Southampton,no,True,False
...,...,...,...,...,...,...,...,...,...
886,male,27.0,13.0000,Second,NaN,Southampton,no,True,False
887,female,19.0,30.0000,First,B,Southampton,yes,True,False
888,female,NaN,23.4500,Third,NaN,Southampton,no,False,True
889,male,26.0,30.0000,First,C,Cherbourg,yes,True,False


In [69]:
# 클래스 별 평균 금액 목적 그냥 클래스별로 DF 생성
First = titaDF[titaDF['class']=='First']
Second = titaDF[titaDF['class']=='Second']
Third = titaDF[titaDF['class']=='Third']

In [57]:
print(f"1등급 평균 금액: {int(First['fare'].mean())}, 2등급 평균 금액 : {int(Second['fare'].mean())}, 3등급 평균 금액 : {int(Third['fare'].mean())}")

1등급 평균 금액: 84, 2등급 평균 금액 : 20, 3등급 평균 금액 : 13


In [93]:
# Second['fare'].describe()
# 클래스 별 금액 describe 합병... 
test = pd.DataFrame(First['fare'].describe())
test1 = pd.DataFrame(Second['fare'].describe())
test2 = pd.DataFrame(Third['fare'].describe())
class_fare = pd.concat([test,test1,test2], axis=1, keys=['First','Second','Third'])
class_fare 

,First,Second,Third
,fare,fare,fare
count,216.000000,184.000000,491.000000
mean,84.154687,20.662183,13.675550
std,78.380373,13.417399,11.778142
min,0.000000,0.000000,0.000000
25%,30.923950,13.000000,7.750000
50%,60.287500,14.250000,8.050000
75%,93.500000,26.000000,15.500000
max,512.329200,73.500000,69.550000


In [95]:
First[(First['fare'] >= 500)]
# Second[(Second['fare']>= 70)]
# Third[(Third['fare']>= 60)]

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
258,female,30s,0,0,512.3292,First,NaN,Cherbourg,yes,True
679,male,30s,0,1,512.3292,First,B,Cherbourg,yes,False
737,male,30s,0,0,512.3292,First,B,Cherbourg,yes,True


In [97]:
titaDF[ (titaDF['class']=='First') & (titaDF['embark_town']=='Cherbourg') & (titaDF['parch']==1) ]
# & (titaDF['sibsp']<1)

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
54,male,60s,0,1,61.9792,First,B,Cherbourg,no,False
97,male,20s,0,1,63.3583,First,D,Cherbourg,yes,False
118,male,20s,0,1,247.5208,First,B,Cherbourg,no,False
155,male,50s,0,1,61.3792,First,NaN,Cherbourg,no,False
273,male,30s,0,1,29.7000,First,C,Cherbourg,no,False
299,female,50s,0,1,247.5208,First,B,Cherbourg,yes,False
319,female,40s,1,1,134.5000,First,E,Cherbourg,yes,False
329,female,teens,0,1,57.9792,First,B,Cherbourg,yes,False
523,female,40s,0,1,57.9792,First,B,Cherbourg,yes,False
581,female,30s,1,1,110.8833,First,C,Cherbourg,yes,False


In [82]:
# 아이들 요금 
kids = titaDF[ (titaDF['age']=='teens') | (titaDF['age']=='baby')]
kids

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
7,male,baby,3,1,21.0750,Third,NaN,Southampton,no,False
9,female,teens,1,0,30.0708,Second,NaN,Cherbourg,yes,False
10,female,baby,1,1,16.7000,Third,G,Southampton,yes,False
14,female,teens,0,0,7.8542,Third,NaN,Southampton,no,True
16,male,baby,4,1,29.1250,Third,NaN,Queenstown,no,False
...,...,...,...,...,...,...,...,...,...,...
855,female,teens,0,1,9.3500,Third,NaN,Southampton,yes,False
869,male,baby,1,1,11.1333,Third,NaN,Southampton,yes,False
875,female,teens,0,0,7.2250,Third,NaN,Cherbourg,yes,True
877,male,teens,0,0,7.8958,Third,NaN,Southampton,no,True


In [91]:
# kids['fare'].describe()

(kids[kids['class']=='First'])['fare'].describe()

kids_fare = pd.concat([(kids[kids['class']=='First'])['fare'].describe(), (kids[kids['class']=='Second'])['fare'].describe() , (kids[kids['class']=='Third'])['fare'].describe()],keys=['kids_F','kids_S','kids_T'], axis=1 )
kids_fare

# 아이들 요금

,kids_F,kids_S,kids_T
count,21.000000,35.000000,108.000000
mean,116.136705,24.725834,17.685765
std,71.882773,12.824884,11.716240
min,26.283300,10.500000,0.000000
25%,57.979200,13.000000,7.917700
50%,108.900000,26.000000,13.935450
75%,151.550000,29.535400,27.900000
max,263.000000,73.500000,46.900000


In [94]:
pd.concat([class_fare, kids_fare], axis=1)

,"(First, fare)","(Second, fare)","(Third, fare)",kids_F,kids_S,kids_T
count,216.000000,184.000000,491.000000,21.000000,35.000000,108.000000
mean,84.154687,20.662183,13.675550,116.136705,24.725834,17.685765
std,78.380373,13.417399,11.778142,71.882773,12.824884,11.716240
min,0.000000,0.000000,0.000000,26.283300,10.500000,0.000000
25%,30.923950,13.000000,7.750000,57.979200,13.000000,7.917700
50%,60.287500,14.250000,8.050000,108.900000,26.000000,13.935450
75%,93.500000,26.000000,15.500000,151.550000,29.535400,27.900000
max,512.329200,73.500000,69.550000,263.000000,73.500000,46.900000


In [60]:
# First[(First['fare']>= 95) & (First['fare']< 200)]
# 33명

In [61]:
# First[(First['fare']>= 200) & (First['fare']< 350)]
# 17명 나옴. 많다.

In [62]:
# First[First['fare']>= 350]
# Cherbourg 탑승 단 3명, 근데 특이하게 비슷한 나이의 3명에 deck와 금액이 모두 동일하다. 셋 다 살았는데 가족은 다른 것이 관계가 없나?

In [63]:
# First[(First['embark_town']=='Cherbourg') & (First['fare'] > 200)]

In [64]:
First[(First['fare'] > 200)]['embark_town'].value_counts()

embark_town
Cherbourg      12
Southampton     8
Name: count, dtype: int64

In [65]:
Second[(Second['fare'] > 60)]['embark_town'].value_counts()

embark_town
Southampton    7
Name: count, dtype: int64

In [66]:
# 퀸즈타운은 왜 이렇게 없나.
titaDF[titaDF['embark_town']=='Queenstown'].describe()

# 퀸즈 타운 1등석도 포함인데 제일 비싼게 90, 물가 차이인가 뭐야 ?

,sibsp,parch,fare
count,77.000000,77.000000,77.000000
mean,0.428571,0.168831,13.276030
std,0.979105,0.656999,14.188047
min,0.000000,0.000000,6.750000
25%,0.000000,0.000000,7.750000
50%,0.000000,0.000000,7.750000
75%,0.000000,0.000000,15.500000
max,4.000000,5.000000,90.000000


##### 티켓등급과 생존율

In [102]:
First   #총 216명.
First[First['alive']=='yes']    # 생존 136명

# 생존율 62퍼

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
1,female,30s,1,0,71.2833,First,C,Cherbourg,yes,False
3,female,30s,1,0,53.1000,First,C,Southampton,yes,False
11,female,50s,0,0,26.5500,First,C,Southampton,yes,True
23,male,20s,0,0,35.5000,First,A,Southampton,yes,True
31,female,NaN,1,0,146.5208,First,B,Cherbourg,yes,False
...,...,...,...,...,...,...,...,...,...,...
862,female,40s,0,0,25.9292,First,D,Southampton,yes,True
871,female,40s,1,1,52.5542,First,D,Southampton,yes,False
879,female,50s,0,1,83.1583,First,C,Cherbourg,yes,False
887,female,teens,0,0,30.0000,First,B,Southampton,yes,True


In [104]:
Second   #총 184명.
# Second[Second['alive']=='yes']    # 생존 87명
# 생존율 47퍼

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
9,female,teens,1,0,30.0708,Second,NaN,Cherbourg,yes,False
15,female,50s,0,0,16.0000,Second,NaN,Southampton,yes,True
17,male,NaN,0,0,13.0000,Second,NaN,Southampton,yes,True
20,male,30s,0,0,26.0000,Second,NaN,Southampton,no,True
21,male,30s,0,0,13.0000,Second,D,Southampton,yes,True
...,...,...,...,...,...,...,...,...,...,...
866,female,20s,1,0,13.8583,Second,NaN,Cherbourg,yes,False
874,female,20s,1,0,24.0000,Second,NaN,Cherbourg,yes,False
880,female,20s,0,1,26.0000,Second,NaN,Southampton,yes,False
883,male,20s,0,0,10.5000,Second,NaN,Southampton,no,True


In [107]:
Third   #총 491명.
# Third[Third['alive']=='yes']    # 생존 119명
# 생존율 24퍼

,sex,age,sibsp,parch,fare,class,deck,embark_town,alive,alone
0,male,20s,1,0,7.2500,Third,NaN,Southampton,no,False
2,female,20s,0,0,7.9250,Third,NaN,Southampton,yes,True
4,male,30s,0,0,8.0500,Third,NaN,Southampton,no,True
5,male,NaN,0,0,8.4583,Third,NaN,Queenstown,no,True
7,male,baby,3,1,21.0750,Third,NaN,Southampton,no,False
...,...,...,...,...,...,...,...,...,...,...
882,female,20s,0,0,10.5167,Third,NaN,Southampton,no,True
884,male,20s,0,0,7.0500,Third,NaN,Southampton,no,True
885,female,30s,0,5,29.1250,Third,NaN,Queenstown,no,False
888,female,NaN,1,2,23.4500,Third,NaN,Southampton,no,False


: 

In [67]:
# 왜 가격 max는 지 혼자 저리 쭉 치솟는가.... deck와 연관이 있나?
deckDF = titaDF[['fare','class','deck','embark_town']]
deckDF
# deck 자료가 너무 없다.

,fare,class,deck,embark_town
0,7.2500,Third,NaN,Southampton
1,71.2833,First,C,Cherbourg
2,7.9250,Third,NaN,Southampton
3,53.1000,First,C,Southampton
4,8.0500,Third,NaN,Southampton
...,...,...,...,...
886,13.0000,Second,NaN,Southampton
887,30.0000,First,B,Southampton
888,23.4500,Third,NaN,Southampton
889,30.0000,First,C,Cherbourg
